In [22]:
#COMPLES FILTERS 

import requests
import json
import pandas as pd
from io import StringIO

fields = [
    "file_name",
    "cases.submitter_id",
    "cases.samples.sample_type",
    "cases.disease_type",
    "cases.project.project_id"
    ]

fields = ",".join(fields)

files_endpt = "https://api.gdc.cancer.gov/files"

# This set of filters is nested under an 'and' operator.
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.primary_site",
            "value": ["Breast"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.experimental_strategy",
            "value": ["RNA-Seq"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        }
    ]
}

# A POST is used, so the filter parameters can be passed directly as a Dict object.
params = {
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "2000"
    }

# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))


s=str(response.content,'utf-8')

data = StringIO(s) 

df=pd.read_csv(data, sep='\t', lineterminator='\r')
print(df.id.head(15).tolist())
display(df.id)

['e7a77848-0df8-4260-b7e9-df8587e9f9b7', '1837ad2a-4edf-4d80-9050-f78115e54454', 'b9610459-bd3e-4d65-92cd-8eb34541f259', '7061c88c-3a5b-4a6b-bf55-bcf20d89a6a7', '21dd15c2-099d-422f-9f05-cfd1170d5bfc', 'a159a8ba-a1e7-4ab6-8820-d67a9c897457', 'c6921a78-f687-46b5-9de7-0603a4a0e99c', 'caaf288e-7672-46af-bc68-302afa0c6478', '54dbdf55-a610-4967-808a-7c4e36178595', 'a62fbc8b-c366-4205-82c0-2a864a2eca87', '048e5e34-ab11-43d6-9788-771502d99152', '1d4fd1cb-6538-4325-bddc-2ced7f437f13', 'c56d71fe-e3cb-4c7f-b464-db7e8b26f71d', 'e05c95c8-bf99-4e0a-bcc6-07e9a5db3c3e', 'ad3b3339-8a4a-4a58-8439-7b82544acec8']


0       e7a77848-0df8-4260-b7e9-df8587e9f9b7
1       1837ad2a-4edf-4d80-9050-f78115e54454
2       b9610459-bd3e-4d65-92cd-8eb34541f259
3       7061c88c-3a5b-4a6b-bf55-bcf20d89a6a7
4       21dd15c2-099d-422f-9f05-cfd1170d5bfc
5       a159a8ba-a1e7-4ab6-8820-d67a9c897457
6       c6921a78-f687-46b5-9de7-0603a4a0e99c
7       caaf288e-7672-46af-bc68-302afa0c6478
8       54dbdf55-a610-4967-808a-7c4e36178595
9       a62fbc8b-c366-4205-82c0-2a864a2eca87
10      048e5e34-ab11-43d6-9788-771502d99152
11      1d4fd1cb-6538-4325-bddc-2ced7f437f13
12      c56d71fe-e3cb-4c7f-b464-db7e8b26f71d
13      e05c95c8-bf99-4e0a-bcc6-07e9a5db3c3e
14      ad3b3339-8a4a-4a58-8439-7b82544acec8
15      dbaba045-5bf8-47b7-b883-ff97abe9a8df
16      bc43acfb-2c72-402b-9be9-d42139c182c1
17      ef184941-92be-4c12-9d56-125d605e9cce
18      6416384b-2b9d-473b-a7e6-016a1df4ed29
19      104d21c5-4d9e-4c81-9399-914c81faa8ad
20      1ffd88db-f671-4886-8daa-b291ecad58f3
21      f45b27f2-b1c3-46f1-8cbb-4f9a22676c18
22      82

In [24]:
import requests
import json
import re

data_endpt = "https://api.gdc.cancer.gov/data"

ids = df.id.head(15).tolist()

params = {"ids": ids}

response = requests.post(data_endpt,
                        data = json.dumps(params),
                        headers={
                            "Content-Type": "application/json"
                            })

response_head_cd = response.headers["Content-Disposition"]

file_name = re.findall("filename=(.+)", response_head_cd)[0]

with open(file_name, "wb") as output_file:
    output_file.write(response.content)

In [26]:
file_uuid_list = ids

data_endpt = "https://api.gdc.cancer.gov/data"

params = {"ids": file_uuid_list}

response = requests.post(data_endpt, data = json.dumps(params), headers = {"Content-Type": "application/json"})

response_head_cd = response.headers["Content-Disposition"]

file_name = re.findall("filename=(.+)", response_head_cd)[0]

with open(file_name, "wb") as output_file:
    output_file.write(response.content)

In [34]:
import zipfile
import os
import fnmatch


def archive1():
    rootPath= (r'/home/CIN/tcnl/Documents/TCGA_API/gdc_download_20180828_160410.670153.tar.gz')
    pattern = '*.tar.gz'
    for root, dirs, files in os.walk(rootPath):
         for filename in fnmatch.filter(files,pattern):
            with zipfile.ZipFile(os.path.join(root, filename)) as zf:
                extractor(zf)

def extractor(zip_file):
    new_dr = r'/home/CIN/tcnl/Documents/TCGA_API'
    extensions = ('.txt')
    [zip_file.extract(file,new_dr) for file in zip_file.namelist() if file.endswith(extensions)]
    print("DONE")


0
1
2
3


In [38]:
import shutil
import glob
import os
import tarfile

with tarfile.open('/home/CIN/tcnl/Documents/TCGA_API/gdc_download_20180828_160410.670153.tar.gz', 'r:gz') as tar:
    tar.extractall()
   # os.chdir('sample')
    for arch in glob.glob('*'):
        shutil.unpack_archive(arch, '/home/CIN/tcnl/Documents/TCGA_API/')

ReadError: Unknown archive format '54dbdf55-a610-4967-808a-7c4e36178595'